# 00 — Quickstart MedVis

This notebook demonstrates:
1. Reading a DICOM series into a 3D volume (SimpleITK)
2. Inspecting orthogonal slices (axial/coronal/sagittal)
3. Extracting an isosurface with marching cubes and saving a mesh (PLY)

**Prerequisites**: Install the project in editable mode: `pip install -e .`
and ensure runtime deps are available (SimpleITK, pyvista, scikit-image, nibabel).

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pyvista as pv
from skimage.measure import marching_cubes

# Project helpers (installed package)
from medvis.io.dicom_series import read_series_to_volume, get_spacing_xyz

# Configure paths
DICOM_DIR = r"data/raw/dicom/assignment_4"  # <-- change to your actual series folder
OUT_DIR = r"reports/figures/quickstart"
os.makedirs(OUT_DIR, exist_ok=True)
print("DICOM_DIR =", DICOM_DIR)
print("OUT_DIR   =", OUT_DIR)

In [ ]:
img = read_series_to_volume(DICOM_DIR)
sx, sy, sz = get_spacing_xyz(img)
print("Spacing (mm):", (sx, sy, sz))

vol = sitk.GetArrayFromImage(img)  # (z, y, x)
print("Volume shape (z,y,x):", vol.shape)

In [ ]:
z, y, x = vol.shape
axial = vol[z // 2, :, :]
coronal = vol[:, y // 2, :]
sagital = vol[:, :, x // 2]

fig, axs = plt.subplots(1, 3, figsize=(12, 4))

axs[0].imshow(axial, cmap="gray")
axs[0].set_title("Axial")

axs[1].imshow(coronal, cmap="gray")
axs[1].set_title("Coronal")

axs[2].imshow(sagital, cmap="gray")  # si tu variable se llama 'sagital', usa ese nombre
axs[2].set_title("Sagittal")

for ax in axs:
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# Choose level heuristically; bones in CT often around 300+ HU (adjust as needed)
level = 300.0
verts, faces, _, _ = marching_cubes(vol, level=level)
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3), faces]).astype(np.int64)
mesh = pv.PolyData(verts[:, ::-1], faces_pv)  # (x,y,z)

ply_path = os.path.join(OUT_DIR, "isosurface.ply")
mesh.save(ply_path)
print("Saved:", ply_path, " | points:", mesh.n_points, "faces:", mesh.n_faces)

In [ ]:
p = pv.Plotter(off_screen=True)
p.add_mesh(mesh, show_edges=False)
p.add_axes()
p.show_bounds(grid="back", location="outer", ticks="outside")
png_path = os.path.join(OUT_DIR, "isosurface_preview.png")
p.show(screenshot=png_path)
print("Saved preview:", png_path)